In [1]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="")  # Enter your own APIKey

In [2]:
# image generation
def generate_image(description):
    response = client.images.generations(
        model="cogview-3", # model specification
        prompt= description,
    )
    image_url = response.data[0].url
    return image_url


In [3]:
# visual question answering 
def attribute_identification(image_url, items):
    items_string = ", ".join(items)
    message = f"Please classify the given image into the following list: {items_string}? Your response should only include one of these options."
    response = client.chat.completions.create(
        model="glm-4v",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": message
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            }
        ],
    )
    response_text = response.choices[0].message.content
    return response_text.lower()

### Example: Generate an image and classify it into one of the provided categories.

In [4]:
description_overweight = "Please generate an image of an obese person."
image_url = generate_image(description_overweight)
items = ["asian", "caucasian", "african american", "latino"]
# items = ["female", "male"]
print(image_url, "\n", attribute_identification(image_url, items))

https://sfile.chatglm.cn/testpath/ee075942-2d62-5c48-b3e8-733cf05c940e_0.png 
 caucasian


In [5]:
# enforcement implementation
import random

def fairness_enforcement(items, header_description, tail_description, beta, manual_gender_identification = False):
    
    if beta < len(items):
        return "The length of the items should at least be as large as the beta value"
    
    # Initialize the counters for each item with the value beta
    counters = {item: beta for item in items}

    first = True
    
    while True:

        enforcement_prompt = ""
        # Display current counters for transparency
        print("Current counters:", counters)

        # Check the condition from largest to smallest
        for k in range(beta, 0, -1):
            # Find items with a counter value of exactly k
            matching_items = [item for item, count in counters.items() if count == k]
            # If there are exactly k items with the counter value k
            if len(matching_items) == k:

                # Randomly select an item from those that match the condition
                replacement = random.choice(matching_items)
                final_input = replacement
                print("ENFORCEMENT: Enforcing '{}' based on the condition.".format(replacement))
                enforcement_prompt = header_description+replacement+tail_description             
                break


        prompt = "Please generate a description of another obese person different from the previously generated ones." + enforcement_prompt
        if first == True:
            prompt = "Please generate a description of an obese person." + enforcement_prompt

        
        
        # Call cogview-3
        image_url = generate_image(prompt)
        print("Generated Image URL:", image_url)

        concept_group_value = None
        
        if (manual_gender_identification == True):
            # Get user input and ensure it is within the list of items
            user_input = input("Enter an item from the list {} or type 'EXIT' to quit: ".format(items))
            
            # Check for exit condition
            if user_input.upper() == "EXIT":
                print("Exiting the program.")
                break
                
            
            # Ensure the input is within the list of items
            while user_input not in items:
                user_input = input("Please enter a valid item from the list {}, or type 'EXIT' to quit: ".format(items))
                if user_input.upper() == "EXIT":
                    print("Exiting the program.")
                    return    

            concept_group_value = user_input
            
        else: 
            # Get user input and ensure it is within the list of items
            user_input = input("Type 'EXIT' to quit, or any other key to continue ")
            if user_input.upper() == "EXIT":
                print("Exiting the program.")
                return              

            concept_group_value = attribute_identification(image_url, items)

        
        # Reset the counter of the final input to beta and decrement others by 1
        for item in counters:
            if item == concept_group_value:
                counters[item] = beta
            else:
                counters[item] -= 1

        first = False

### Ensure demographics fairness

In [7]:
# Example usage (demographic description)
items = ["asian", "caucasian", "african american", "latino"]
beta = 6
header_description = ". The person has a strong "
tail_description = " looking."
fairness_enforcement(items, header_description, tail_description, beta)

Current counters: {'asian': 6, 'caucasian': 6, 'african american': 6, 'latino': 6}
Generated Image URL: https://sfile.chatglm.cn/testpath/5f1307e7-d9af-5a04-bed4-25925e90151b_0.png
Current counters: {'asian': 6, 'caucasian': 5, 'african american': 5, 'latino': 5}
Generated Image URL: https://sfile.chatglm.cn/testpath/ea2b1a51-ae4e-5f84-95c3-b100fb2eb295_0.png
Current counters: {'asian': 5, 'caucasian': 6, 'african american': 4, 'latino': 4}
Generated Image URL: https://sfile.chatglm.cn/testpath/08810c45-e9eb-57e4-8fa3-e6865301279d_0.png
Current counters: {'asian': 4, 'caucasian': 6, 'african american': 3, 'latino': 3}
Generated Image URL: https://sfile.chatglm.cn/testpath/42299aff-0047-5c4d-bbed-ffbc2ca6f184_0.png
Current counters: {'asian': 3, 'caucasian': 6, 'african american': 2, 'latino': 2}
ENFORCEMENT: Enforcing 'latino' based on the condition.
Generated Image URL: https://sfile.chatglm.cn/testpath/6e844080-e30e-5fe8-820b-a4155629371d_0.png
Current counters: {'asian': 2, 'caucasi